## UAFLP Formulations by Konak et al. (2006)

In [ ]:
import numpy as np
from docplex.mp.model import Model

### Data Instances:

In [ ]:
# Input data: Instance O7
n_dpts = 7
max_bays = np.arange(1, 10)
dpts = np.arange(1, n_dpts + 1)
# width = 13
# height = 8.54
width = 8.54
height = 13
dp_areas = np.array([16, 16, 16, 36, 9, 9, 9])
asp_ratio = np.array([4]*n_dpts)
max_side = np.minimum(np.array([height]*n_dpts), np.sqrt(asp_ratio*dp_areas))
min_side = np.sqrt(dp_areas/asp_ratio)
mat_flows = np.array([[0,  0,  0,  0,  0,  0,  0],
                      [0,  0,  0,  0,  0,  0,  0],
                      [0,  0,  0,  0,  0,  0,  0],
                      [5,  3,  2,  0,  0,  0,  0],
                      [0,  0,  0,  4,  0,  0,  0],
                      [0,  0,  0,  4,  0,  0,  0],
                      [1,  1,  1,  0,  2,  1,  0]])

mat_flows = mat_flows.transpose()

In [ ]:
# Input data: Instance O8
n_dpts = 8
max_bays = np.arange(1, 10)
dpts = np.arange(1, n_dpts + 1)
width = 11.31
height = 13
dp_areas = np.array([16, 16, 16, 36, 36, 9, 9, 9])
asp_ratio = np.array([4]*n_dpts)
max_side = np.minimum(np.array([height]*n_dpts), np.sqrt(asp_ratio*dp_areas))
min_side = np.sqrt(dp_areas/asp_ratio)
mat_flows = np.array([[0,  0,  0,  0,  0,  0,  0,  0],
                      [0,  0,  0,  0,  0,  0,  0,  0],
                      [0,  0,  0,  0,  0,  0,  0,  0],
                      [5,  3,  2,  0,  0,  0,  0,  0],
                      [5,  3,  2,  0,  0,  0,  0,  0],
                      [0,  0,  0,  4,  3,  0,  0,  0],
                      [0,  0,  0,  4,  0,  0,  0,  0],
                      [1,  1,  1,  0,  4,  2,  1,  0]])

mat_flows = mat_flows.transpose()

In [ ]:
# Input data: Instance O9
n_dpts = 9
max_bays = np.arange(1, 10)
dpts = np.arange(1, n_dpts + 1)
width = 12
height = 13
dp_areas = np.array([16, 16, 16, 36, 36, 9, 9, 9, 9])
asp_ratio = np.array([4]*n_dpts)
max_side = np.minimum(np.array([height]*n_dpts), np.sqrt(asp_ratio*dp_areas))
min_side = np.sqrt(dp_areas/asp_ratio)
mat_flows = np.array([[0,  0,  0,  0,  0,  0,  0,  0,  0],
                      [0,  0,  0,  0,  0,  0,  0,  0,  0],
                      [0,  0,  0,  0,  0,  0,  0,  0,  0],
                      [5,  3,  2,  0,  0,  0,  0,  0,  0],
                      [5,  3,  2,  0,  0,  0,  0,  0,  0],
                      [0,  0,  0,  4,  3,  0,  0,  0,  0],
                      [0,  0,  0,  4,  0,  0,  0,  0,  0],
                      [0,  0,  0,  0,  0,  0,  0,  0,  0],
                      [1,  1,  1,  0,  4,  2,  1,  0,  0]])

mat_flows = mat_flows.transpose()

### MIP-FSBP Formulation

In [ ]:
# Define UAFLP-MIP model
uaflp_mip = Model('UAFLP-MIP-KONAK-2006')

# Add variables
zs = {(i, k): uaflp_mip.binary_var(name=f'z_{i}_{k}')\
      for i in dpts for k in max_bays}

rs = {(i, j): uaflp_mip.binary_var(name=f'r_{i}_{j}')\
      for i in dpts for j in dpts}

ds = {(k): uaflp_mip.binary_var(name=f'd_{k}')\
      for k in max_bays}

w = {(k): uaflp_mip.continuous_var(name=f'w_{k}', lb=0)\
      for k in max_bays}

ly = {(i): uaflp_mip.continuous_var(name=f'l_y_{i}', lb=0)\
      for i in dpts}

h = {(i, k): uaflp_mip.continuous_var(name=f'h_{i}_{k}', lb=0)\
       for i in dpts for k in max_bays}

x = {(i): uaflp_mip.continuous_var(name=f'x_{i}', lb=0)\
      for i in dpts}

y = {(i): uaflp_mip.continuous_var(name=f'y_{i}', lb=0)\
      for i in dpts}

dx = {(i, j): uaflp_mip.continuous_var(name=f'd_x_{i}_{j}', lb=0)\
        for i in dpts for j in dpts}

dy = {(i, j): uaflp_mip.continuous_var(name=f'd_y_{i}_{j}', lb=0)\
        for i in dpts for j in dpts}

In [ ]:
# Add objective function
# total_mf_costs = uaflp_mip.linear_expr()

# for i in dpts:
#     for j in dpts:
#         if i < j:
#             print(i, j, mat_flows[i-1, j-1])
#             total_mf_costs.add_term(dx[i, j], mat_flows[i-1, j-1])
#             total_mf_costs.add_term(dy[i, j], mat_flows[i-1, j-1])


total_mf_costs = uaflp_mip.sum(uaflp_mip.sum(mat_flows[i-1, j-1] * (dx[i, j] + dy[i, j]) for j in dpts if i < j) for i in dpts)

uaflp_mip.minimize(total_mf_costs)

In [ ]:
# Add constraints
for i in dpts:
    for j in dpts:
        if i < j:
            uaflp_mip.add_constraint(dx[i, j] >= x[i] - x[j])
            uaflp_mip.add_constraint(dx[i, j] >= x[j] - x[i])
            uaflp_mip.add_constraint(dy[i, j] >= y[i] - y[j])
            uaflp_mip.add_constraint(dy[i, j] >= y[j] - y[i])

In [ ]:
for i in dpts:
    uaflp_mip.add_constraint(uaflp_mip.sum(zs[i, k] for k in max_bays) == 1)

In [ ]:
for k in max_bays:
    uaflp_mip.add_constraint(w[k] == uaflp_mip.sum(zs[i, k] * dp_areas[i-1] for i in dpts)/height)

In [ ]:
for i in dpts:
    for k in max_bays:
        uaflp_mip.add_constraint(min_side[i-1] * zs[i, k] <= w[k])
        uaflp_mip.add_constraint(w[k] <= max_side[i-1] + width * (1 - zs[i, k]))

In [ ]:
for i in dpts:
    for k in max_bays:
        uaflp_mip.add_constraint(x[i] >= uaflp_mip.sum(w[j] for j in max_bays if j <= k) - 0.5*w[k] - \
                                 (width - min_side[i-1])*(1 - zs[i, k]))
        
        uaflp_mip.add_constraint(x[i] <= uaflp_mip.sum(w[j] for j in max_bays if j <= k) - 0.5*w[k] + \
                                 (width - min_side[i-1])*(1 - zs[i, k]))

In [ ]:
for i in dpts:
    for j in dpts:
        if i < j:
            for k in max_bays:
                uaflp_mip.add_constraint(h[i, k]/dp_areas[i-1] - h[j, k]/dp_areas[j-1] - \
                                         np.max([max_side[i-1]/dp_areas[i-1], max_side[j-1]/dp_areas[j-1]])*(2 - zs[i, k] - zs[j, k]) <= 0)
                uaflp_mip.add_constraint(h[i, k]/dp_areas[i-1] - h[j, k]/dp_areas[j-1] + \
                                         np.max([max_side[i-1]/dp_areas[i-1], max_side[j-1]/dp_areas[j-1]])*(2 - zs[i, k] - zs[j, k]) >= 0)

In [ ]:
for k in max_bays:
    uaflp_mip.add_constraint(uaflp_mip.sum(h[i, k] for i in dpts) == height*ds[k])

In [ ]:
for i in dpts:
    for k in max_bays:
        uaflp_mip.add_constraint(min_side[i-1]*zs[i, k] <= h[i, k])
        uaflp_mip.add_constraint(h[i, k] <= max_side[i-1]*zs[i, k])

In [ ]:
for i in dpts:
    uaflp_mip.add_constraint(uaflp_mip.sum(h[i, k] for k in max_bays) == ly[i])

In [ ]:
for i in dpts:
    for j in dpts:
        if i != j:
            uaflp_mip.add_constraint(y[i] - 0.5*ly[i] >= y[j] + 0.5*ly[j] - height*(1 - rs[i, j]))

In [ ]:
for i in dpts:
    for j in dpts:
        if i < j:
            uaflp_mip.add_constraint(rs[i, j] + rs[j, i] <= 1)

In [ ]:
for i in dpts:
    for j in dpts:
        if i < j:
            for k in max_bays:
                uaflp_mip.add_constraint(rs[i, j] + rs[j, i] >= zs[i, k] + zs[j, k] - 1)

In [ ]:
for i in dpts:
    uaflp_mip.add_constraint(0.5*ly[i] <= y[i])
    uaflp_mip.add_constraint(y[i] <= height - 0.5*ly[i])

In [ ]:
uaflp_mip.pprint()

In [ ]:
uaflp_sol = uaflp_mip.solve(log_output=True)

In [ ]:
uaflp_sol.get_objective_value()

* O7(v1):

    - Width = 13
    - Height = 8.54
    - OPT: 136.58016035347748

* O7(v2):

    - Width = 8.54
    - Height = 13
    - OPT: 134.19007457625085

* O8:

    - Width = 11.31
    - Height = 13
    - OPT: 245.50564435564434
    - Sol:
    ```
    z_1_9  =  1.0
    z_2_9  =  1.0
    z_3_1  =  1.0
    z_4_9  =  1.0
    z_5_1  =  1.0
    z_6_1  =  1.0
    z_7_9  =  1.0
    z_8_1  =  1.0
    --
    r_1_2  =  1.0
    r_3_5  =  1.0
    r_3_6  =  1.0
    r_3_8  =  1.0
    r_4_1  =  1.0
    r_4_2  =  1.0
    r_6_5  =  1.0
    r_6_8  =  1.0
    r_7_1  =  1.0
    r_7_2  =  1.0
    r_7_4  =  1.0
    r_8_5  =  1.0
    --
    (x, y)  =  (8.346153846153847, 4.0519480519480515)
    (x, y)  =  (8.346153846153847, 1.3506493506493504)
    (x, y)  =  (2.6923076923076925, 11.514285714285714)
    (x, y)  =  (8.346153846153847, 8.441558441558442)
    (x, y)  =  (2.6923076923076925, 3.342857142857143)
    (x, y)  =  (2.6923076923076925, 9.192857142857143)
    (x, y)  =  (8.346153846153847, 12.24025974025974)
    (x, y)  =  (2.6923076923076925, 7.521428571428572)
    ```

* O9:

    - Width = 12
    - Height = 13
    - OPT: 254.3873303167421
    - Sol:
    ```
    z_1_3  =  1.0
    z_2_1  =  1.0
    z_3_9  =  1.0
    z_4_3  =  1.0
    z_5_1  =  1.0
    z_6_1  =  1.0
    z_7_9  =  1.0
    z_8_9  =  1.0
    z_9_1  =  1.0
    --
    r_2_1  =  1.0
    r_2_5  =  1.0
    r_2_6  =  1.0
    r_2_9  =  1.0
    r_4_1  =  1.0
    r_6_5  =  1.0
    r_6_9  =  1.0
    r_7_3  =  1.0
    r_8_3  =  1.0
    r_8_7  =  1.0
    r_9_5  =  1.0
    --
    (x, y)  =  (7.384615384615385, 2.0)
    (x, y)  =  (2.6923076923076925, 11.514285714285714)
    (x, y)  =  (10.692307692307693, 3.058823529411764)
    (x, y)  =  (7.384615384615385, 8.5)
    (x, y)  =  (2.6923076923076925, 3.3428571428571434)
    (x, y)  =  (2.6923076923076925, 9.192857142857143)
    (x, y)  =  (10.692307692307693, 7.838235294117647)
    (x, y)  =  (10.692307692307693, 11.279411764705882)
    (x, y)  =  (2.6923076923076925, 7.521428571428572)
    
    ```


In [ ]:
for i in dpts:
    for k in max_bays:

        zs_val = uaflp_sol.get_var_value(zs[i, k])
        if zs_val == 1.0:
            print(f'z_{i}_{k}', ' = ', zs_val)

In [ ]:
for i in dpts:
    for j in dpts:

        rs_val = uaflp_sol.get_var_value(rs[i, j])
        if rs_val == 1.0:
            print(f'r_{i}_{j}', ' = ', rs_val)

In [ ]:
for i in dpts:

    x_coord = uaflp_sol.get_var_value(x[i])
    y_coord = uaflp_sol.get_var_value(y[i])
    print('(x, y)', ' = ', (x_coord, y_coord))

### Best FSBP Model

MIP-FSBP model + constraints (23)-(27)

In [ ]:
relaxed = True

In [ ]:
# Original MIP-FBS Formulation
# Define UAFLP-MIP model
uaflp_mip = Model('UAFLP-MIP-KONAK-2006')

# Add variables
# Binary vars
if not relaxed:
    zs = {(i, k): uaflp_mip.binary_var(name=f'z_{i}_{k}')\
        for i in dpts for k in max_bays}

    rs = {(i, j): uaflp_mip.binary_var(name=f'r_{i}_{j}')\
        for i in dpts for j in dpts}

    ds = {(k): uaflp_mip.binary_var(name=f'd_{k}')\
        for k in max_bays}
else:
    zs = {(i, k): uaflp_mip.continuous_var(name=f'z_{i}_{k}', lb=0)\
      for i in dpts for k in max_bays}

    rs = {(i, j): uaflp_mip.continuous_var(name=f'r_{i}_{j}', lb=0)\
        for i in dpts for j in dpts}

    ds = {(k): uaflp_mip.continuous_var(name=f'd_{k}', lb=0)\
        for k in max_bays}

# Continuous vars
w = {(k): uaflp_mip.continuous_var(name=f'w_{k}', lb=0)\
      for k in max_bays}

ly = {(i): uaflp_mip.continuous_var(name=f'l_y_{i}', lb=0)\
      for i in dpts}

h = {(i, k): uaflp_mip.continuous_var(name=f'h_{i}_{k}', lb=0)\
       for i in dpts for k in max_bays}

x = {(i): uaflp_mip.continuous_var(name=f'x_{i}', lb=0)\
      for i in dpts}

y = {(i): uaflp_mip.continuous_var(name=f'y_{i}', lb=0)\
      for i in dpts}

dx = {(i, j): uaflp_mip.continuous_var(name=f'd_x_{i}_{j}', lb=0)\
        for i in dpts for j in dpts}

dy = {(i, j): uaflp_mip.continuous_var(name=f'd_y_{i}_{j}', lb=0)\
        for i in dpts for j in dpts}

# Add objective function
total_mf_costs = uaflp_mip.sum(uaflp_mip.sum(mat_flows[i-1, j-1] * (dx[i, j] + dy[i, j]) for j in dpts if i < j) for i in dpts)
uaflp_mip.minimize(total_mf_costs)

# Add constraints
for i in dpts:
    for j in dpts:
        if i < j:
            uaflp_mip.add_constraint(dx[i, j] >= x[i] - x[j])
            uaflp_mip.add_constraint(dx[i, j] >= x[j] - x[i])
            uaflp_mip.add_constraint(dy[i, j] >= y[i] - y[j])
            uaflp_mip.add_constraint(dy[i, j] >= y[j] - y[i])

for i in dpts:
    uaflp_mip.add_constraint(uaflp_mip.sum(zs[i, k] for k in max_bays) == 1)

for k in max_bays:
    uaflp_mip.add_constraint(w[k] == uaflp_mip.sum(zs[i, k] * dp_areas[i-1] for i in dpts)/height)

for i in dpts:
    for k in max_bays:
        uaflp_mip.add_constraint(min_side[i-1] * zs[i, k] <= w[k])
        uaflp_mip.add_constraint(w[k] <= max_side[i-1] + width * (1 - zs[i, k]))

for i in dpts:
    for k in max_bays:
        uaflp_mip.add_constraint(x[i] >= uaflp_mip.sum(w[j] for j in max_bays if j <= k) - 0.5*w[k] - \
                                 (width - min_side[i-1])*(1 - zs[i, k]))
        
        uaflp_mip.add_constraint(x[i] <= uaflp_mip.sum(w[j] for j in max_bays if j <= k) - 0.5*w[k] + \
                                 (width - min_side[i-1])*(1 - zs[i, k]))
        
for i in dpts:
    for j in dpts:
        if i < j:
            for k in max_bays:
                uaflp_mip.add_constraint(h[i, k]/dp_areas[i-1] - h[j, k]/dp_areas[j-1] - \
                                         np.max([max_side[i-1]/dp_areas[i-1], max_side[j-1]/dp_areas[j-1]])*(2 - zs[i, k] - zs[j, k]) <= 0)
                uaflp_mip.add_constraint(h[i, k]/dp_areas[i-1] - h[j, k]/dp_areas[j-1] + \
                                         np.max([max_side[i-1]/dp_areas[i-1], max_side[j-1]/dp_areas[j-1]])*(2 - zs[i, k] - zs[j, k]) >= 0)

for k in max_bays:
    uaflp_mip.add_constraint(uaflp_mip.sum(h[i, k] for i in dpts) == height*ds[k])

for i in dpts:
    for k in max_bays:
        uaflp_mip.add_constraint(min_side[i-1]*zs[i, k] <= h[i, k])
        uaflp_mip.add_constraint(h[i, k] <= max_side[i-1]*zs[i, k])

for i in dpts:
    uaflp_mip.add_constraint(uaflp_mip.sum(h[i, k] for k in max_bays) == ly[i])

for i in dpts:
    for j in dpts:
        if i != j:
            uaflp_mip.add_constraint(y[i] - 0.5*ly[i] >= y[j] + 0.5*ly[j] - height*(1 - rs[i, j]))

for i in dpts:
    for j in dpts:
        if i < j:
            uaflp_mip.add_constraint(rs[i, j] + rs[j, i] <= 1)

for i in dpts:
    for j in dpts:
        if i < j:
            for k in max_bays:
                uaflp_mip.add_constraint(rs[i, j] + rs[j, i] >= zs[i, k] + zs[j, k] - 1)

for i in dpts:
    uaflp_mip.add_constraint(0.5*ly[i] <= y[i])
    uaflp_mip.add_constraint(y[i] <= height - 0.5*ly[i])

In [ ]:
# uaflp_mip.pprint()

In [ ]:
# Solve original model
uaflp_sol = uaflp_mip.solve(clean_before_solve=True, log_output=True)

In [ ]:
# Show solution for the original model
print(uaflp_sol.solve_status)
print(uaflp_mip.solve_details.time)
print(uaflp_mip.solve_details.nb_nodes_processed)
print(uaflp_sol.get_objective_value())

In [ ]:
for i in dpts:
    for k in max_bays:

        zs_val = uaflp_sol.get_var_value(zs[i, k])
        if zs_val > 0:
            print(f'z_{i}_{k}', ' = ', zs_val)

In [ ]:
for i in dpts:
    for j in dpts:

        dx_val = uaflp_sol.get_var_value(dx[i, j])
        dy_val = uaflp_sol.get_var_value(dy[i, j])
        if dx_val != 0 or dy_val != 0:
            print(dx_val, dy_val)

#### Adding improvements to the model

In [ ]:
# Tightening LP lower bound
# for i in dpts:
#     for j in dpts:
#         if i < j:
#             for k in max_bays:
#                 for m in max_bays:
#                     if k != m:
#                         uaflp_mip.add_constraint(dx[i, j] >= 0.5*(min_side[i-1] + min_side[j-1]) + width*(zs[i, k] + zs[j, m] - 2))

for i in dpts:
    for j in dpts:
        if i < j:
            for k in max_bays:
                uaflp_mip.add_constraint(dy[i, j] >= 0.5*(min_side[i-1] + min_side[j-1]) + height*(zs[i, k] + zs[j, k] - 2))

In [ ]:
# Reducing model degeneracy: fill bays sequentially
for k in max_bays:
    if k < len(max_bays):
        uaflp_mip.add_constraint(n_dpts*uaflp_mip.sum(zs[i, k] for i in dpts) >= uaflp_mip.sum(zs[i, k+1] for i in dpts))

In [ ]:
# Tightened dx constraint
for i in dpts:
    for j in dpts:
        if i < j:
            for k in max_bays:
                for m in max_bays:
                    if k != m:
                        uaflp_mip.add_constraint(dx[i, j] >= 0.5*(min_side[i-1] + min_side[j-1]) + \
                                                 uaflp_mip.min(min_side[p-1] for p in dpts if p != i and p != j)*(np.abs(m - k) - 1) + \
                                                    len(max_bays)*width*(zs[i, k] + zs[j, m] - 2))

In [ ]:
# Solve original model + improvements
uaflp_sol = uaflp_mip.solve(clean_before_solve=True, log_output=True)

In [ ]:
# Show solution for the original model
print(uaflp_sol.solve_status)
print(uaflp_mip.solve_details.time)
print(uaflp_mip.solve_details.nb_nodes_processed)
print(uaflp_sol.get_objective_value())

In [ ]:
for i in dpts:
    for k in max_bays:

        zs_val = uaflp_sol.get_var_value(zs[i, k])
        if zs_val > 0:
            print(f'z_{i}_{k}', ' = ', zs_val)

In [ ]:
uaflp_mip.add_constraint(zs[1, 1] == 1)
uaflp_mip.add_constraint(zs[2, 1] == 1)
uaflp_mip.add_constraint(zs[3, 1] == 1)
uaflp_mip.add_constraint(zs[4, 2] == 1)
uaflp_mip.add_constraint(zs[5, 2] == 1)
uaflp_mip.add_constraint(zs[6, 2] == 1)
uaflp_mip.add_constraint(zs[7, 1] == 1)

uaflp_mip.add_constraint(rs[1, 2] == 1)
uaflp_mip.add_constraint(rs[1, 3] == 1)
uaflp_mip.add_constraint(rs[1, 7] == 1)
uaflp_mip.add_constraint(rs[2, 3] == 1)
uaflp_mip.add_constraint(rs[2, 7] == 1)
uaflp_mip.add_constraint(rs[3, 7] == 1)
uaflp_mip.add_constraint(rs[4, 5] == 1)
uaflp_mip.add_constraint(rs[4, 6] == 1)
uaflp_mip.add_constraint(rs[5, 6] == 1)

uaflp_mip.add_constraint(d[4, 6] == 1)
uaflp_mip.add_constraint(rs[5, 6] == 1)